In [1]:
from datetime import datetime, date
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from random import random
import time
import re
# current version of seaborn generates a bunch of warnings that we'll ignore
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Use this to test for captcha block or IP ban
def get_URL(position,location):
    #from torrequest import TorRequest
    """[Build a template url for a dummy call to verify the site isn't returning a captcha]
    Args:
        position ([string]): [job for query]
        location ([string]): [location for query]
    Returns:
        [string]: [formatted url]
    """
    template = 'https://www.indeed.com/jobs?q={}&l={}&fromage=1&sort=date'
                
    position = position.replace(' ', '%20')
    location = location.replace(' ', '+')
    url = template.format(position,location)
    return url


# from torrequest import TorRequest
# tr=TorRequest(password='your_super_secure_password')
position = 'data scientist'
location = 'california'
# tr.reset_identity()
response = requests.get(get_URL(position,location))
# This will either return an HTML block for a captcha or of a search result
#response

In [3]:
response

<Response [200]>

In [ ]:

soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
searchResults = soup.find('div', id='mosaic-provider-jobcards')


In [ ]:
refinedsearchResults = searchResults.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})

In [ ]:
len(refinedsearchResults)

In [ ]:
z = searchResults.children
lst = []
for i in z:
    lst.append(i)


In [ ]:

x = lst[0]


checking again

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
#searchResults = soup.find('div', id='mosaic-provider-jobcards')
refinedsearchResults = soup.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})
        

raw_posts = []
for post in refinedsearchResults:
        raw_posts.append(post)
        n = 0


In [ ]:
z = raw_posts[0]

In [ ]:
url = z.find('a', href=True)
url


In [ ]:

z.find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']

In [ ]:
z

In [ ]:
postDate = z.find('span', 'date').text
extractDate = datetime.today().strftime('%Y-%m-%d')
summary = z.find('div', 'job-snippet').text.strip().replace('\n', ' ')


In [ ]:
summary

In [ ]:
company_name = z.find('a', attrs={'class':'turnstileLink companyOverviewLink'}).text.strip()
company_name

In [ ]:
job_title = z.find('a', attrs={'class':'jcs-JobTitle'}).text.strip()
job_title

In [5]:
def get_desc_features(job_url):
    """Parses each job description, searching for and extracting values for features

    Args:
        job_url (string): http address of each job posting

    Returns:
        tuple: job feature values
    """
    response_job_desc = requests.get(job_url)
    soup = BeautifulSoup(response_job_desc.text, 'html.parser')

    
    try:
        salary_and_jType = soup.find('div', id='salaryInfoAndJobType').text.strip()
    except:
        salary_and_jType = None
    if salary_and_jType == None:
        try:
            salary_and_jType = soup.find('div',id="icl-u-xs-block jobsearch-ReqAndQualSection-item--title").text.replace("\n", "").strip()
        except:
            salary_and_jType = None
    #TODO get benefits from its designated section
    
    
    try:
        sal_guide_items = []
        items = soup.find('ul',class_='css-1lyr5hv eu4oa1w0')
        for i in items:
            sal_guide_items.append(i.text)
    except:
        sal_guide_items = None
        
        
    try:
        salfromsection = soup.find('span',class_='icl-u-xs-mr--xs').text
    except:
        salfromsection = None
        
        
    try:
        job_type_items = []
        job_type_from_section = soup.find('div',class_='jobsearch-JobDescriptionSection-sectionItem').next_sibling.children
        for i in job_type_from_section:
            if i.text == 'Job Type':
                continue
            else:
                job_type_items.append(i.text)
    except:
        job_type_items = None
        
    
    try:
        requirements = soup.find(class_="icl-u-xs-block jobsearch-ReqAndQualSection-item--title").text.replace("\n", "").strip()      

    except:
        requirements = None
        
        
    try:
        description = soup.find(id="jobDescriptionText").text.replace('\n', '')
    except:
        description = None
        
        
    # A nifty little workaround for evading detection.
    time.sleep(.5+random()*3)
    #TODO assess h2 tags commonalities to determine if these section descriptions are from Indeed or are at least of only a few variations.
        #you could then distinguish the description into sections and conduct NLP etc each.
    raw_desc_soup = soup
    return salary_and_jType, sal_guide_items, salfromsection, job_type_items, requirements, description, raw_desc_soup

In [12]:
#TODO condense these with lists, particularly fields that have .text.strip()
def get_features(post):
    """parses search results and extracts basic job feature values,
        then combines this with output of 'get_desc_features' function.

    Args:
        post (string): response for each post in search results page

    Returns:
        dict: single-feature deep dictionary of features (dictionary keys) and their values (dictionary values)
    """
    datapoint_dict = {}

    title = post.find('h2',
              attrs={'class': lambda e: e.startswith('jobTitle') if e else False}).text.replace('new', '')

    company = post.find('span', 'companyName').text.strip()
    try:
        rating = post.find('span', 'ratingNumber').text
    except:
        rating = None

    location = post.find('div', 'companyLocation').text.strip()
    postDate = post.find('span', 'date').text
    extractDate = datetime.today().strftime('%Y-%m-%d')
    summary = post.find('div', 'job-snippet').text.strip().replace('\n', ' ')
    url = 'https://www.indeed.com'+ post.find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']

    try:
        estimated_salary = post.find('span','estimated-salary').text.strip()
    except:
        estimated_salary = None
    try:
        salary = post.find('div','metadata salary-snippet-container').text.strip()
    except:
        salary = None


        
            
    salary_and_jType, sal_guide_items, salfromsection, job_type_items, requirements, description, raw_desc_soup = get_desc_features(url)
    datapoint_dict = {
                        'title':title,
                        'company':company,
                        'rating':rating,
                        'location':location,
                        'salary':salary,
                        'estimated_salary':estimated_salary,
                        'postDate':postDate,
                        'extractDate':extractDate,
                        'summary':summary,
                        'url':url,
                        'salary_and_jType':salary_and_jType,
                        'sal_guide_items':sal_guide_items,
                        'salfromsection':salfromsection,
                        'job_type_items':job_type_items,
                        'requirements':requirements,
                        'description':description,
                        'raw_desc_soup':raw_desc_soup}
    if len(datapoint_dict) > 0:
        return datapoint_dict
    else:
        pass

In [7]:
def main(position, location):
    """[Conducts the web scraping process]
    Args:
        position ([string]): [job position for indeed.com query]
        position ([string]): [job location for indeed.com query]
        
        Returns:
        [csv]: [scraped data]
    """
    data = pd.DataFrame()
    
    # extract the job data
    while True:
        response = requests.get(get_URL(position, location))
        soup = BeautifulSoup(response.text, 'html.parser')
        searchResults = soup.find('div', id='mosaic-provider-jobcards')
        refinedsearchResults = searchResults.find_all('div', attrs={'class': lambda e: e.startswith('cardOutline') if e else False})
        

        raw_posts = []
        for post in refinedsearchResults:
            raw_posts.append(post)
        
        n = 0
        for post in raw_posts:
            datapoint = get_features(post)
            data = data.append(datapoint, ignore_index=True)
        # Again, a nifty little workaround for evading detection.
            n+=1
            print(n)
            
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except:
            break

    name = position.replace(' ','_')
    loc = location.replace(' ','_')
    day = date.today()
    # save the job data
    data.to_csv(f'../app/data/scraped_{name}_{loc}_{day}.csv', index=False)
    return raw_posts

In [18]:
state_names = [ "alabama", "arkansas",  "arizona", "california", "colorado", "connecticut", "delaware", "florida", "georgia", "remote", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia",  "vermont", "washington", "wisconsin", "west virginia", "wyoming"]
for state in state_names:
    position = 'data scientist'
    location = state
    print(state)
    data = main(position,location )

alabama
1
arkansas
1
arizona
1
california
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
colorado
1
2
connecticut
1
2
3
4
5
delaware
1
2
florida
1
georgia
1
2
3
remote
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
iowa
1
idaho
1
illinois
1
2
3
4
5
6
7
8
9
10
indiana
1
kansas
1
kentucky
1
louisiana
1
2
massachusetts
1
2
3
4
5
6
7
8
9
10
maryland
1
2
3
4
5
6
7
maine
1
michigan
1
2
3
minnesota
1
missouri
1
mississippi
1
montana
1
north carolina
1
2
3
4
5
6
north dakota
1
nebraska
1
2
new hampshire
1
new jersey
1
2
3
4
new mexico
1
nevada
1
new york
1
2
3
4
5
6
7
8
9
10
ohio
1
2
3
4
5
6
7
8
oklahoma
1
oregon
1
pennsylvania
1
2
3
4
rhode island
1
south carolina
1
south dakota
1
tennessee
1
2
3
4
5
texas
1
2
3
4
5
6
7
8
9
10
utah

In [16]:
position = 'data scientist'
location = 'california'
data = main(position,location )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
name = position.replace(' ','_')
loc = location.replace(' ','_')
day = date.today()
data.to_csv(f'../app/data/scraped_{name}_{loc}_{day}.csv', index=False)

The  below is used for various adjustments to my webscraping process.

### Concatenating Old Data With New

In [49]:
lst = []
records = 0
for i in state_names:
    i = i.replace(' ','_')
    location = pd.read_csv(f'../app/data/scraped_data_scientist_{i}_2022-05-31.csv')
    records += len(location)
    lst.append(location)

print(records)

282


In [ ]:

total = pd.read_csv(f'../app/data/total.csv')
lst.append(total)

In [29]:
#total = pd.read_csv(f'../app/data/total.csv')

z = pd.concat(lst)
z.to_csv('../app/data/total.csv', index= False)

In [54]:
x = pd.concat([z,total])
x.to_csv('../app/data/total.csv', index= False)

In [52]:
total

,company,description,estimated_salary,extractDate,job_type_items,location,postDate,rating,raw_desc_soup,requirements,sal_guide_items,salary,salary_and_jType,salfromsection,summary,title,url
0,Online Technical Services,"Job descriptionData Scientist, MarketingSAN DI...",NaN,2022-04-13,['Full-time'],Remote,PostedJust posted,3.7,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",Master's (Preferred)Python: 1 year (Preferred)...,NaN,"$145,000 - $150,000 a year","$145,000 - $150,000 a year - Full-time","$145,000 - $150,000 a year",Identify relevant data sources and data sets t...,Data Scientist - Marketing,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,West CAP,HUMAN was founded in 2012 in a Brooklyn sci-fi...,Estimated $114K – $144K a year,2022-04-13,NaN,"Remote in New York, NY+2 locations",PostedJust posted,3.5,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$114K to $14...",NaN,Full-time,NaN,You’ve worked as a data scientist solving larg...,"Data Scientist, BotGuard",https://www.indeed.com/rc/clk?jk=58cdde046f643...
2,Maya Ai inc.,Our Maya team is expanding and we are looking ...,NaN,2022-04-13,"['Full-time', 'Part-time']",Remote,PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",Python: 1 year (Preferred)SQL: 1 year (Preferred),NaN,"$77,766 - $183,411 a year","$77,766 - $183,411 a year - Full-time, Part-time","$77,766 - $183,411 a year",Our Analyst will be dealing with data coming i...,Data Scientist,https://www.indeed.com/company/Maya-Ai-inc./jo...
3,"EMERGETECH, INC",Description:Job CategoryData ScienceAbout Emer...,Estimated $94.7K – $120K a year,2022-04-13,NaN,Remote,PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$94.7K to $1...",NaN,NaN,NaN,Design and create the data sources that ”citiz...,Data Scientist,https://www.indeed.com/rc/clk?jk=95fb128bb025f...
4,Recurrent,What's the opportunity?Recurrent is on a missi...,Estimated $119K – $151K a year,2022-04-13,NaN,"Remote in Seattle, WA",PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$119K to $15...",NaN,NaN,NaN,Experienced - you have 2+ years of experience ...,Data Scientist,https://www.indeed.com/rc/clk?jk=e9ce610b72deb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,Lucid Technologies Inc,Job Title: Information Security Data Scientist...,Estimated $95.4K - $121K a year,2022-04-20,NaN,"Remote in Dallas, TX",PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$95.4K - $12...",NaN,NaN,NaN,Participate as a key member of the analytics a...,Information Security Data Scientist (Remote),https://www.indeed.com/rc/clk?jk=6202e64a38d75...
1511,Windstream Communications,We are looking for a data scientist to help us...,NaN,2022-04-20,NaN,Remote,PostedJust posted,3.2,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,NaN,Full-time,NaN,You will also be responsible for acquisition o...,Senior Data Scientist,https://www.indeed.com/rc/clk?jk=5362358221574...
1512,Edmunds.com,At Edmunds we’re driven to make car buying eas...,Estimated $47.5K - $60.1K a year,2022-04-20,NaN,"Remote in Santa Monica, CA 90404",PostedJust posted,3.4,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$47.5K to $6...",NaN,Full-time,NaN,The senior analyst/data scientist will serve a...,Senior Analyst/Data Scientist,https://www.indeed.com/rc/clk?jk=9dcaee4b15174...
1513,One Concern,About One Concern One Concern brings disaster ...,Estimated $118K - $150K a year,2022-04-20,NaN,Remote,PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$118K - $150...",NaN,Full-time,NaN,Collaborate with an interdisciplinary team of ...,"Senior Data Scientist, Wind Resilience",https://www.indeed.c

In [53]:
total.extractDate.unique()

array(['2022-04-13', '2022-04-14', '2022-04-15', '2022-04-17',
       '2022-04-19', '2022-04-20'], dtype=object)

In [31]:
z

,company,description,estimated_salary,extractDate,job_type_items,location,postDate,rating,raw_desc_soup,requirements,sal_guide_items,salary,salary_and_jType,salfromsection,summary,title,url
0,S&P Global,Panjiva is a data-driven technology company th...,NaN,2022-05-31,['Full-time'],United States,PostedToday,3.9,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$91,500 - $190,100 a year","$91,500 - $190,100 a year - Full-time","$91,500 - $190,100 a year",Working with our data scientists to turn large...,Data Engineer - Data Science Platforms & Infra...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
0,S&P Global,Panjiva is a data-driven technology company th...,NaN,2022-05-31,['Full-time'],United States,PostedToday,3.9,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$91,500 - $190,100 a year","$91,500 - $190,100 a year - Full-time","$91,500 - $190,100 a year",Working with our data scientists to turn large...,Data Engineer - Data Science Platforms & Infra...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
0,CyberCoders,Data Scientist If you are a Data Scient...,NaN,2022-05-31,['Full-time'],"Remote in Phoenix, AZ 85021",PostedToday,3.6,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$100,000 - $110,000 a year","$100,000 - $110,000 a year - Full-time","$100,000 - $110,000 a year",Collaborate with analytics teams to create ins...,Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
0,Standard Chartered,Job: Data and Analytics Primary Location: As...,Estimated $105K - $133K a year,2022-05-31,NaN,"San Francisco, CA 94105 (Financial District/So...",PostedJust posted,4.1,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,"['', 'Not provided by employer', ""$105K - $133...",NaN,Full-time,NaN,Develop machine learning and advanced analytic...,Specialist Data Scientist,https://www.indeed.com/rc/clk?jk=0c043ee62c575...
1,Primary Talent Partners,Primary Talent Partner has an exciting 12 mont...,NaN,2022-05-31,"['Full-time', 'Contract']","San Jose, CA",PostedJust posted,NaN,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",Bachelor's (Required)US work authorization (Re...,NaN,$65 - $70 an hour,"$65 - $70 an hour - Full-time, Contract",$65 - $70 an hour,"Understanding of statistical modeling, machine...",Sr. Data Analyst,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,CyberCoders,Remote Unity AI Engineer If you are a R...,NaN,2022-05-31,['Full-time'],"Remote in Seattle, WA 98109",PostedToday,3.6,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$100,000 - $200,000 a year","$100,000 - $200,000 a year - Full-time","$100,000 - $200,000 a year","Experience with creating modding tools, or any...",Remote Unity AI Engineer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,Amazon.com Services LLC,PhD degree with 4 years of applied research ex...,NaN,2022-05-31,NaN,"Seattle, WA",Posted1 day ago,3.5,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,NaN,Full-time,NaN,The Model accelerator and Data platform is loo...,"Applied Science Manager, Model Accelerator and...",https://www.indeed.com/rc/clk?jk=f088d36c6064a...
0,S&P Global,Panjiva is a data-driven technology company th...,NaN,2022-05-31,['Full-time'],United States,PostedToday,3.9,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$91,500 - $190,100 a year","$91,500 - $190,100 a year - Full-time","$91,500 - $190,100 a year",Working with our data scientists to turn large...,Data Engineer - Data Science Platforms & Infra...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
0,S&P Global,Panjiva is a data-driven technology company th...,NaN,2022-05-31,['Full-time'],United States,PostedToday,3.9,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"">\...",NaN,NaN,"$91,500 - $190,100 a year","$91,500 - $190,100 a year - Full-time","$91,500 - $190,100 a year",Working with our data scientists to turn large...,Data E

In [ ]:
#fix old imports

data['extractDate']= pd.to_datetime(data['extractDate'])

def pDate(row):
    from datetime import datetime, date, timedelta

    #days_ago = row['dateposted']
    delta = timedelta(0)
    try:
        return row['extractDate'] - delta
    except:
        return row

data['extractDate'] = data.apply( lambda row : pDate(row), axis = 1)
data['extractDate'] = data['extractDate'].astype(str)

In [ ]:
data.to_csv('../app/data/scraped_data_scientist_remote_2022-04-14.csv', index= False)

In [ ]:
data.extractDate.unique()